# ABOUT


Datascientest's Datascientist continuous bootcamp - cohorte Mars2022 -  AeroBOT project

**Tutor**

* Alban THUET

**Authors:**

* Hélène ASSIR
* Hichem HADJI  
* [Ioannis STASINOPOULOS](https://www.linkedin.com/in/ioannis-stasinopoulos/)

</br>

---
</br>

**Version History**

Version | Date       | Author(s)  | Modification
--------|----------- | ---------  | --------------------------
X.X     | XX/XX/2022 | A.B        | modif
1.1     | 04/07/2022 | I.S, H.A.  | Anomaly one-hot encoding
1.0     | 30/06/2022 | I.S, H.A.  | Document creation

This notebook can be executed entirely. 

It

* mounts the GDrive of our AeroBot project @gmail account.

* loads the data from the `train_data_final.pkl` file (cf. `0_test_set_creator_DO_NOT_MODIFY_20220630.ipynb`) under a pandas DataFrame named `df`, which contains `97417` entries and 96 columns.
These data do not contain any more UAS-related entries.

* Performs Feature Engineering: one-hot-encoding of Anomaly using a naive approache, looking directly for the root label into the multiple modalities string of Anomaly
* A comparison of 2 approaches (Naive vs Regex, more complex) was performed, but, with very limited discrepancies between the 2 approaches, we favored the naive one (see : ATTEMPT_1_main_DO_NOT_MODIFY_20220704_Anomaly_ReGex.ipynb).  

Further processing of the data should be perfomed on this dataframe, after having **copied a version of the present notebook**.

# IMPORT PACKAGES


settings for  full / patial Narrative display. Helene?


In [2]:
#######################
# Import packages
#######################
import numpy as np
import seaborn as sns

#######################
# Pandas
#######################
import pandas as pd
# Set pandas settings to show all data when using .head(), .columns etc.
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option("display.colheader_justify","left") # left-justify the print output of pandas

### Display full columnwidth
# Set pandas settings to display full text columns
#pd.options.display.max_colwidth = None
# Restore pandas settings to display standard colwidth
pd.reset_option('display.max_colwidth')

import itertools # Pour créer des iterateurs
######################
# TIME
from time import time
###################### 
######################
# PLOTTING
######################
import matplotlib.pyplot as plt
%matplotlib inline
# Define global plot parameters for better readability and consistency among plots
# A complete list of the rcParams keys can be retrieved via plt.rcParams.keys() function
plt.rcParams['axes.titlesize'] = 30
plt.rcParams['axes.labelsize'] = 23
plt.rcParams['xtick.labelsize'] = 23
plt.rcParams['ytick.labelsize'] = 23
plt.rc('legend', fontsize=23)    # legend fontsize

# BOKEH 
from bokeh.plotting import figure # Importation de la classe figure qui permet de créer un graphique bokeh.
from bokeh.io import  push_notebook, output_notebook, show
output_notebook() # permet d'afficher tous les futurs graphiques dans l'output d'une cellule jupyter. Si cette instruction n'est pas lancée, la figure s'affichera dans un nouvel onglet.
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
from bokeh.models.tools import HoverTool

#####################
# NLP 
#####################
import re # for Regular Expression handling
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet') # WordNet lemmatizer
nltk.download('omw-1.4') # necessary for WordNet lemmatizer
from nltk.tokenize import word_tokenize # Usual tokenizer
from nltk.tokenize import TweetTokenizer # Special tokenizer;  "we'll", "didn't", etc. are considered as one word
from nltk.tokenize import PunktSentenceTokenizer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer # Vectorization
from nltk.corpus import stopwords # Import stopwords from nltk.corpus
###############################
# Clustering
###############################
from sklearn.cluster import  KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from scipy.spatial.distance import cdist
###############################
# ML preprocessing and models
###############################
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import classification_report, confusion_matrix

import pickle as pkl # Saving data externally

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


# LOAD DATA

## Mount GDrive

In [3]:
#@title
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive/')

#check your present working directory 
%pwd

Mounted at /content/drive/


'/content'

In [4]:
#@title
# move to the transformed data location (you can create a deeper structure, if needed, e.g. to save a trained model):
%cd /content/drive/MyDrive/data/transformed/

/content/drive/MyDrive/data/transformed


In [5]:
#@title
!ls # list the content of the pwd

#!ls "/content/drive/MyDrive/Data_Science/Formations/DataScienceTest/projet/AeroBot/" # list contect of a speficic folder

'Copy of Qualified abbreviations_20220718.xlsx.gsheet'
'Data Dictionnary.xlsx'
 Narrative_PP_stemmed_21072022_TRAIN.pkl
 Narrative_Raw_Stemmed_21072022_TRAIN.pkl
 Narrative_RegEx_subst_21072022_TRAIN.pkl
'Qualified abbreviations_20220707_test.csv'
'Qualified abbreviations_20220708.csv'
'Qualified abbreviations_20220718.csv'
'Qualified abbreviations_20220718_Google_sheet.gsheet'
 test_data_final.pkl
 train_data_final.pkl


## Load data from .pkl file


In [6]:
#@title
# Load the TRAIN data (97417 entries)
# Do not touch the TEST data until the end of the project!
# or the curse of the greek gods will fall upon you!

%cd /content/drive/MyDrive/data/transformed/
with open("train_data_final.pkl", "rb") as f:
    loaded_data = pkl.load(f)

df = loaded_data[0]
print("\nA Dataframe with", len(df), "entries has been loaded")

/content/drive/MyDrive/data/transformed

A Dataframe with 97417 entries has been loaded


# FEATURE ENGINEERING

## Anomaly

This section 

*   drops the entries with NaN in 'Anomaly'
*   creates and adds to the DataFrame 'df' a one-hot column for each Anomaly root label

Root labels are as defined in the ASRS coding form.pdf

We use a naive approach using string parsing 

### Drop the NaNs in Anomaly

In [7]:
# Drop Anomaly NaN's
print(f"We have {len(df[df['Anomaly'].isna()])} entries where 'Anomaly' == NaN. We drop these entries")

df = df.dropna(axis = 0, how = 'any', subset = ['Anomaly'])
print(f"Current length of our DataFrame: {len(df)}")

We have 431 entries where 'Anomaly' == NaN. We drop these entries
Current length of our DataFrame: 96986


### One-hot encode Anomaly root labels

In [8]:
# Root label (source = ASRS coding forms)
Anomaly_RootLabels=['Aircraft Equipment',
                    'Airspace Violation',
                    'ATC Issue',
                    'Flight Deck / Cabin / Aircraft Event',
                    'Conflict',
                    'Deviation - Altitude',
                    'Deviation - Speed',
                    'Deviation - Track / Heading',
                    'Deviation / Discrepancy - Procedural',
                    'Ground Excursion',
                    'Ground Incursion',
                    'Ground Event / Encounter',
                    'Inflight Event / Encounter',
                    'No Specific Anomaly Occurred']

#### Naive one-hot encoding

In [9]:
# Create a column in the df corresponding to each Anomaly root label 'anomaly_rl'
for anomaly_rl in Anomaly_RootLabels:
    col='Anomaly_'+anomaly_rl
    
    # Fill the columns in a one-hot-encoding logic
    df[col]=df['Anomaly'].astype('str').apply(lambda x: 1 if (anomaly_rl in x)  else 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
for anomaly_rl in Anomaly_RootLabels:
    col='Anomaly_'+anomaly_rl
    print('\n',col,'\n',df[col].value_counts())


 Anomaly_Aircraft Equipment 
 0    59054
1    37932
Name: Anomaly_Aircraft Equipment, dtype: int64

 Anomaly_Airspace Violation 
 0    92989
1     3997
Name: Anomaly_Airspace Violation, dtype: int64

 Anomaly_ATC Issue 
 0    81232
1    15754
Name: Anomaly_ATC Issue, dtype: int64

 Anomaly_Flight Deck / Cabin / Aircraft Event 
 0    89839
1     7147
Name: Anomaly_Flight Deck / Cabin / Aircraft Event, dtype: int64

 Anomaly_Conflict 
 0    79622
1    17364
Name: Anomaly_Conflict, dtype: int64

 Anomaly_Deviation - Altitude 
 0    88110
1     8876
Name: Anomaly_Deviation - Altitude, dtype: int64

 Anomaly_Deviation - Speed 
 0    94398
1     2588
Name: Anomaly_Deviation - Speed, dtype: int64

 Anomaly_Deviation - Track / Heading 
 0    88707
1     8279
Name: Anomaly_Deviation - Track / Heading, dtype: int64

 Anomaly_Deviation / Discrepancy - Procedural 
 1    58337
0    38649
Name: Anomaly_Deviation / Discrepancy - Procedural, dtype: int64

 Anomaly_Ground Excursion 
 0    94829
1     

In [11]:
# Plots
# # Parsing de la colonne 'Anomaly' et top 200 des modalités selon le 1/0 encoding
# for anomaly_rl in Anomaly_RootLabels:
#     col='Anomaly_'+anomaly_rl
    
#     title_plot=col +' : ' + str(sum(df[col]))+' entries of our DF'
#     Anomaly=df[df[col]==1]['Anomaly'].str.split('; ', expand=True).stack().reset_index(drop=True)
#     fig, ax = plt.subplots(figsize=(15,30))
#     sns.countplot(y=Anomaly, order=Anomaly.value_counts().iloc[:50].index,color='green').set(title=title_plot);

#     title_plot='NO '+col +' : ' + str(len(df)-sum(df[col]))+' entries of our DF'
#     Anomaly=df[df[col]==0]['Anomaly'].str.split('; ', expand=True).stack().reset_index(drop=True)
#     fig, ax = plt.subplots(figsize=(15,30))
#     sns.countplot(y=Anomaly, order=Anomaly.value_counts().iloc[:50].index,color='red').set(title=title_plot);

# NARRATIVE - TEXT PREPROCESSING

## Dealing with abbreviations

### Import of abbreviations data : 


In [ ]:
# @ Yannis from Eleni : fichier mis à jour le 8 juillet. Pour ta part (1-letter/unit), j'ai:
# - mis à jour "Mach number" en "Mach"
# - splité en 2 lignes feet et foot
df_abbrev = pd.read_csv('Qualified abbreviations_20220708.csv', low_memory=False,index_col=0, sep=';')

In [ ]:
df_abbrev=df_abbrev[['Abbreviation','Full-Text','Category','Forms to parse','Note']]
df_abbrev.head()

,Abbreviation,Full-Text,Category,Forms to parse,Note
ID,,,,,
1,C,Celsius,1 letter,NaN,NaN
2,E,East,1 letter,NaN,"Split from N,S,E,W"
3,F,Fahrenheit,1 letter,NaN,NaN
4,L,Left,1 letter,NaN,split from L/R
5,M,Mach,1 letter,NaN,NaN


In [ ]:
df_abbrev['Category'].value_counts()

Expression     288
Contraction    144
Unit            20
1 letter        10
Name: Category, dtype: int64

### Counters: document/absolute frequency

In [ ]:
def doc_frequ_counter(word):
  """
  Count the documment frequency (i.e. in how many narratives is the word found) of 'word'
  """
  # Initialize counter
  doc_frequ = 0

  for narr in df['Narrative']:
    # /!\ use the .lower instances of word to seach for and texts to look into
    if word.lower() in narr.lower():
      doc_frequ += 1

  return doc_frequ

In [ ]:
# Create new columns with the counters
# Purpose: get a first impression of the frequencies
# Takes ~ 2min. to execute
df_abbrev['doc_frequ_abb'] = df_abbrev['Abbreviation'].apply(doc_frequ_counter)
# Note that this counter is definitely over-estimated, e.g. 'ALT' is found 
# both as stand-alone expression AND when in 'altitude', 'altimeter' etc.
# Same happens with 'GOV', but *not* with e.g. 'OVCST' (stands for 'overcast')

# Full-text 
df_abbrev['doc_frequ_ft'] = df_abbrev['Full-Text'].apply(doc_frequ_counter)
# This counter should be quite accurate

### All except units and 1-letters

In [ ]:
df_abbrev.sort_values(by = 'doc_frequ_ft', ascending = False)[(df_abbrev['Category'] != '1 letter') & (df_abbrev['Category'] != 'Unit')]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Abbreviation,Full-Text,Category,Forms to parse,Note,doc_frequ_abb,doc_frequ_ft
ID,,,,,,,
31,ACFT,Aircraft,Contraction,NaN,NaN,26390,36483
80,FLT,Flight,Contraction,NaN,NaN,21616,29692
52,CLR,Clear,Contraction,;S;ED;ING,NaN,18334,25871
57,CTL,Control,Contraction,;S;ED;ING,NaN,26292,24088
145,RWY,Runway,Contraction,;S,NaN,19355,22648
131,PLT,Pilot,Contraction,;S;'S,NaN,11933,22170
35,APCH,Approach,Contraction,;S;ED;ING,NaN,14771,21911
108,LNDG,Landing,Contraction,NaN,NaN,13643,20124
50,CHK,Check,Contraction,;S;ED;ING,NaN,13412,19491


### Units and 1-letters

In [ ]:
# These abbreviations require a treatment by RegEx's...
# Prioritize the the ones that have a high doc_frequ_abb
# Don't forget also their respective 'Forms to parse' when existing
df_abbrev.sort_values(by = 'doc_frequ_abb', ascending = False)[(df_abbrev['Category']== '1 letter') | (df_abbrev['Category']== 'Unit')]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,Abbreviation,Full-Text,Category,Forms to parse,Note,doc_frequ_abb,doc_frequ_ft
ID,,,,,,,
2,E,East,1 letter,NaN,"Split from N,S,E,W",97412,9884
6,N,North,1 letter,NaN,"Split from N,S,E,W",97407,6794
7,R,Right,1 letter,NaN,split from L/R,97395,20202
4,L,Left,1 letter,NaN,split from L/R,97369,19441
8,S,South,1 letter,NaN,"Split from N,S,E,W",97342,6497
1,C,Celsius,1 letter,NaN,NaN,97264,128
3,F,Fahrenheit,1 letter,NaN,NaN,97193,47
9,W,West,1 letter,NaN,"Split from N,S,E,W",97060,6638
5,M,Mach,1 letter,NaN,NaN,96867,1075


In [ ]:
# def RegEx_finder(regex):
#   """
#   Search the RegEx passed as input within each narrative.
#   Return all the findings in a list.
#   """
#   findings = []
#   for narr in df['Narrative']:

#     findings = findings + regex.findall(narr)
#   print(f"Found {len(findings)} matches\n\n")
  
#   return findings

In [ ]:
# Write all the entries of 'Anomaly' into a single string using 
# a delimiter which is not present in the data, i.e. '$'
all_narr = ''
all_narr = '$'.join(str(entry) for entry in df['Narrative'][:1000]) # do not include a ' ' before / after the '$' !

#### N, E, S, W

We do not replace these letters, due to ambiguity in their meaning

In [ ]:
# Compile RegEx for cardinal orientations
r_card = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \s                            
                  [NESW]
                  \s                            
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_card.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 49 matches


In [ ]:
findings[-20:]

['he north side of the E Gates. I was watchin',
 'cular taxiways G and N intersecting the 2 p',
 'd he was going to do S turns so he could de',
 "' that emboldens how N is now the Northrout",
 ' re-emphasizing that N is the Northroute co',
 'e airspace was Class E instead of the usual',
 ' think that this MEL s intended to give rel',
 '.3 and asked for the N number of the Navajo',
 'tions (a request for S turns or more timely',
 ' that we use taxiway N as it had been open ',
 'continued to taxiway N and turned off at lo',
 "ng action on taxiway N as 'nil' and advised",
 ' said to use Taxiway N and said something a',
 'tiated a hand off to S tags.  The aircraft ',
 'aware that the Eglin E ATCAA and Restricted',
 'n many Class D/Class E after-hours airfield',
 ' not able to get any N number and was too s',
 'th heard use Taxiway W or V. We had already',
 ' aircraft on taxiway S saw Aircraft X enter',
 'nce the weather. ACR W reported turning to ']

Most expressions HAVE NOTHING TO DO with orientations, e.g.:

*   'Class E' airspace 
*   taxiway E <-- this seems to be the most common



#### Runway indications 'R/L/C'

In [ ]:
# # Compile Regular Expressions
# r_R_L_C = re.compile(r"""
#                   (?i)                          # turns on the case-insensitive mode of RegEx
#                   #.{0,20}                       # show up to 20 characters before the pattern
#                   \d{1,}R 
#                   | 
#                   \d{1,}L                  # at least a number *before* 'R' or 'L'
#                   #.{0,20}                       # show up to 20 characters after the pattern
#                   """, re.VERBOSE)
# findings = r_R_L_C.findall(all_narr)
# print(f"Found {len(findings)} matches")

In [ ]:
# Compile Regular Expressions
r_R_L_C = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \d+R 
                  | 
                  .{0,20}\d+L                  # at least a number *before* 'R' or 'L'
                  |
                  .{0,20}\d+C
                  .{0,20}                       # show up to 20 characters after the pattern
                  """, re.VERBOSE)
findings = r_R_L_C.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 1400 matches


In [ ]:
# Compile Regular Expressions
r_Right_Left_Center = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \d+ Right 
                  | 
                  .{0,20}\d+ Left                  # at least a number *before* 'R' or 'L'
                  |
                  .{0,20}\d+ Center
                  .{0,20}                       # show up to 20 characters after the pattern
                  """, re.VERBOSE)
findings = r_Right_Left_Center.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 0 matches


In [ ]:
findings[:10]

[]

There are also some entries like 'R4', or 'R4L' (number follows the letter); we choose to ignore them for now, since their meaning is not clear.

#### Celcius

replace negative temperatures by smth like 'TEMP_C_NEG' and positive temperatures by 'TEMP_C_POS' 

In [ ]:
# Compile RegEx
r_Celcius_neg = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \s                            # Avoid finding an aircraft model type, e.g. 'MH-65C'
                  -                             # Our hypothesis: most temp. indications are negative
                                                # otherwise confustion with RWY indication '36C' (center)
                  \d+                           
                  C
                  \s                            
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_Celcius_neg.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 42 matches


In [ ]:
findings[:20]

[' at ZZZ overnight in -13c temps and was extrem',
 ' the temperature was -8C (approximately 17 de',
 'T temps were already -10c and thought nothing ',
 'ltitude. SAT was now -27C and the aircraft sti',
 ' the temperature was -18c down to cross YAGER ',
 'h a predicted OAT of -25C did I believe that i',
 'tures would be below -40C (AFM definition of i',
 't; we were told that -70C is the limit for the',
 'he weather and below -15c if possible. Reroute',
 'e temperature was at -17C or below. His refere',
 'e at destination was -15C and the outlook was ',
 ' temperature. It was -25C and I proceeded to b',
 '2;500 feet. Temp was -12c wind was 200 @ 40 kt',
 'IMC at approximately -34C about 10 minutes bef',
 'oted an OAT of about -5C & turned on the Engi',
 ' an INDICATED SAT of -56C the aircraft began a',
 'to AFM limitation of -40C minimum temp for win',
 'ating them below the -40C limitation? If the S',
 'AT falsely indicates -35C but the actual SAT i',
 'ny temperature below -10C was 2

When searching for `\s C \s`

you find e.g. 'Class C' airspace

When searching for 

```
\d+ C \s
```
you find
* RUNWAY indications, e.g. '36C' ('C' stands for 'center')  
* 'seated in 8C' <-- seat indication
* temperature indications tend to appear mostly preceeded by a negative sign



In [ ]:
# Compile RegEx for 'XX degrees C'
r_Celcius_degrees = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \d+
                  \s                           
                  degrees                       # appears mostly in plural form
                  \s* 
                  C
                  \s                            # without this, you will match '120 degree clearing turn' 
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_Celcius_degrees.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 6 matches


In [ ]:
findings[:10]

['ake temperature was 545 degrees C (Centigrade) and the',
 't inboard was again 35 degrees C (it had only changed',
 ' 45; then cooled to 27 degrees C enroute.  By landing',
 'air temperature was 26 degrees C and power levers wer',
 ' in our aircraft at 26 degrees C OAT. This presents a',
 'icating between 750-800 degrees C at around 20% torque']

#### Farenheit

later eventually: check whether 3-digit + F matches  temperature indications unambiguously

In [ ]:
# Compile RegEx
r_F_1 = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \s
                  \d+                           
                  F
                  \s                            
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_F_1.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 7 matches


In [ ]:
findings[:10]

['re on the ground was 61F and fuel tank temper',
 'nder 1;4;& 6 rose to 485F which max temp is 50',
 'd area that sent out 1300F possible flame that ',
 'alling; temps around 10F and we had approxima',
 'rcraft that departed 52F airport westbound wa',
 'h aircraft departing 52F airport westbound. I',
 ' pilot outreach with 52F airport and yet airc']

Can be confused with:
*   seat indication, e.g. 'A passenger at 5F was also concerned'
*   'aircraft departing 52F airport' (what kind of airport designation is this??)



In [ ]:
# Compile RegEx for 'XX degrees F'
r_F_2 = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \d+
                  \s                           
                  degrees                       # appears mostly in plural
                  \s* 
                  F
                  \s                            # without this, you will match '120 degree clearing turn' 
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_F_2.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 1 matches


In [ ]:
findings[:10]

['ne cylinder reading 400 degrees F - not normal! I deci']

#### M

We do not treat this.

In [ ]:
# Compile RegEx
r_M = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \s
                  M
                  \s                        
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_M.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 14 matches


In [ ]:
findings[:10]

[' the new ACARS - REV M had been implemented',
 'to the revision. REV M defaults to using re',
 ' fuel. The ACARS REV M training videos only',
 'e the turn to follow M in front of the Fire',
 'ake the turn back to M and we would be unab',
 'as until I undershot M on M2. I thought we ',
 " to ' Taxi from spot M via North Route 22; ",
 'i them north on L or M there are 17R approa',
 'half of them stop on M or L; but we really ',
 'nuing north to reach M and turning left the']

#### Speed in Mach

In [ ]:
# Compile RegEx - variant 1
r_Mach_1 = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  #.{0,20}                       # show up to 20 characters before the pattern
                  Mach
                  \s*
                  0*\.\d{1,}
                  #.{0,20}                       # show up to 20 characters after the pattern
                  """, re.VERBOSE)
findings = r_Mach_1.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 7 matches


In [ ]:
findings[:20]

['Mach .84',
 'Mach .834',
 'Mach .80',
 'Mach .74',
 'MACH .78',
 'Mach .78',
 'Mach .75']

In [ ]:
# Compile RegEx - variant 2
r_Mach_2 = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  #.{0,20}                       # show up to 20 characters before the pattern
                  0*\.\d{1,}
                  \s*
                  Mach
                  #.{0,20}                       # show up to 20 characters after the pattern
                  """, re.VERBOSE)
findings = r_Mach_2.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 4 matches


In [ ]:
findings[:20]

['.864 mach', '0.78 Mach', '.78 mach', '.79 mach']

##### Mach trim

We choose not to treat this one; it is a good example of an N-gram (like 'yaw damper'). 
Let's see whether our CountVectorizer will list it among the most frequent N-grams...

There are also several mentions of 'Mach trim'


`A Mach trimmer is a device which varies the pitch trim automatically as a function of Mach number to oppose Mach tuck and maintain level flight.`

We could treat this as an expression.

In [ ]:
# Compile RegEx - variant 3
r_Mach_3 = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  Mach\sTrim                  
                  .{0,20}                       # show up to 20 characters after the pattern
                  """, re.VERBOSE)
findings = r_Mach_3.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 8 matches


In [ ]:
findings[-20:]

['that the autopilot; Mach trim and yaw damper were',
 'aged the autopilot; Mach trim and yaw damper.  Ou',
 "ot 'STAB TRIM' and 'MACH TRIM' status messages. I",
 "red the end of the 'MACH TRIM' QRH procedure we g",
 'red normal prior to Mach Trim Inop light on EOAP ',
 'First officer began Mach Trim Inop checklist. Sho',
 'ould call them back.Mach Trim Inop checklist comp',
 'B CH 1 message; and MACH TRIM message. There was ']

#### wind gusts

In [ ]:
# Compile RegEx for KTS indications
r_gust = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \d+
                  g
                  \d+
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_gust.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 19 matches


In [ ]:
findings[:10]

[' [destination] (080-20G30) and our fuel state',
 'sh.$Gusty winds 220@14G24 with the FO (First ',
 'ersonnel. METAR 150/7G14 10SM -RA OVC019 A29',
 'were previously 330/18G25; then 290/18. Toron',
 ' was: [wind] 190 at 16G29 KTS; visibility 10 ',
 'wind conditions 320/19G21 RNAV/GPS [approach]',
 'ecast was for winds 20G35 knots in MAF. Nothi',
 't with winds 010 at 52G67 knots. Clearly the ',
 'd; the wind was 330/20G27KT; so a takeoff on ',
 'reporting winds 260 17G28. The FO (First Offi']

#### Z time

In [ ]:
# Compile RegEx - variant 1
# The letter "Z" ("Zulu") indicates Coordinated Universal Time (UTC).
# 0000Z format
r_Z_1 = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \s
                  \d{4,4}                       # exactly 4 digits preceeding Zulu time (UTC +0) 'Z'    
                  Z
                  \s                            
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_Z_1.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 0 matches


In [ ]:
findings[:10]

[]

In [ ]:
# Compile RegEx - variant 2
# Find the 00:00Z form
r_Z_2 = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \s
                  \d{2,2}                       # exactly 2 digits     
                  \:
                  \d{2,2}
                  Z
                  \s                            
                  .{0,20}                       # show up to 20 characters after the pattern 
                  """, re.VERBOSE)

findings = r_Z_2.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 0 matches


In [ ]:
findings[:10]

[]


The RegEx 
```
\s Z \s
```
returns mostly anonymized designations, e.g. 'aircraft Z', 'mechanic Z' etc.


#### Speed in KTS

In [ ]:
# Compile RegEx for KTS indications
r_KTS = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \d+
                  KTS
                  |
                  \d+
                  KT                        
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_KTS.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 46 matches


In [ ]:
findings[:10]

['crosswind (230 at 8-10kts',
 '22kt; and visibility gre',
 'sked us to maintain 180kts',
 '05KT P6SM VCSH SCT025 BK',
 's approximately 060/15kts',
 ' DYMON and maintain 280kts',
 '260KT with Aircraft X abo',
 '140KT. I issued reduce sp',
 '170KT to Aircraft Y; to w',
 '00000KT 2SM  -RA  BR  BKN00']

#### Pressure hPa

do not replace, due to very few occurences

In [ ]:
# Compile RegEx - variant 1
r_hPa = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \d+
                  \s*
                  hPa    
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)
findings = r_hPa.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 0 matches


In [ ]:
findings[:20]

[]

#### Altitudes


In [ ]:
# Compile RegEx - variant 1
r_alt_1 = re.compile(r"""
                    (?i)                          # turns on the case-insensitive mode of RegEx
                    #.{0,20}                       # show up to 20 characters before the pattern
                    FL\d{1,}     
                    #.{0,20}                       # show up to 20 characters after the pattern 
                    """, re.VERBOSE)
findings = r_alt_1.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 793 matches


In [ ]:
findings[:10]

['FL180',
 'FL350',
 'FL320',
 'Fl320',
 'FL320',
 'FL310',
 'FL270',
 'FL230',
 'FL190',
 'FL320']

In [ ]:
# # Compile RegEx - variant 2
# r_alt_2 = re.compile(r"""
#                   (?i)                          # turns on the case-insensitive mode of RegEx
#                   #.{0,20}                       # show up to 20 characters before the pattern
#                   \d{1,}
#                   \s*
#                   FT
#                   \s*
#                   M*S*L*     
#                   .{0,20}                       # show up to 20 characters after the pattern          
#                 """, re.VERBOSE)
# findings = r_alt_2.findall(all_narr)
# print(f"Found {len(findings)} matches")

In [ ]:
# Compile RegEx - variant 2
r_alt_2 = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  #.{0,20}                       # show up to 20 characters before the pattern
                  \d{0,2}[\.\,\;]
                  \d*
                  FT
                  #.{0,20}                       # show up to 20 characters after the pattern          
                """, re.VERBOSE)
findings = r_alt_2.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 1 matches


This will also find 'This is 3.0 FT long', which is a length indication, not altitude!

Replace ONLY the ones without a space between the number and 'FT' by ' FT ' (with spaces preceeding and following 'FT' !)

We do not search for 'MSL' (Mean Sea Level) that is often present an the end of altitude indications, because it does not bring added value.

In [ ]:
findings[:10]

['10;000ft']

We could split the altitude levels into 3 categories using a different threshold: 


*   high altitude
*   middle altitude
*   low altitude



#### AM/PM Time indications

In [ ]:
# Compile RegEx - variant 2
r_pm = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx                 
                  .{0,20}                       # show up to 20 characters before the pattern
                  [0-1X\w]{1,2}
                  [\;\:\,\.]*
                  [0-59]{1,2}
                  pm                
                  .{0,20}                       # show up to 20 characters after the pattern          
                """, re.VERBOSE)

findings = r_pm.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 7 matches


In [ ]:
findings[:10]

[' to the gate around 1:30pm. As the crew and I ',
 'o depart to ZZZZ at XB:45pm.  In addition to th',
 ']. At approximately XC:00pm I was made aware of',
 'ernight; duty on at XK:05PM; 5 hours block; 8 h',
 ' then end my day at XQ:30PM at another overnigh',
 'assigned. At around XA:05pm approach advised us',
 'our-day finished at XA30pm. I then had to take']

In [ ]:
# Compile RegEx - variant 3
r_am = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx                 
                  .{0,20}                       # show up to 20 characters before the pattern
                  [0-1X\w]{1,2}
                  [\;\:\,\.]*
                  [0-59]{1,2}
                  am                
                  .{0,20}                       # show up to 20 characters after the pattern          
                   """, re.VERBOSE)

findings = r_am.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 11 matches


can be confused with 'amps/amps' for Ampères --> we will replace by ' am' (NO space at the end). This way amps will stay amps and will be treated correctly by the tokenizer.

##### AM without num

In [12]:
# Write all the entries of 'Anomaly' into a single string using 
# a delimiter which is not present in the data, i.e. '$'
narr_2009 = ''
narr_2009 = '$'.join(str(entry) for entry in df[df['Year']>=2009]['Narrative']) # do not include a ' ' before / after the '$' !

In [16]:
# Compile RegEx
r_AM_2009 = re.compile(r"""
                  .{0,20}                       # show up to 20 characters before the pattern
                  [\s\.\,\;\:\/]
                  AM
                  [\s\.\,\;\:\/]
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_AM_2009.findall(narr_2009)
print(f"Found {len(findings)} matches")

Found 138 matches


In [19]:
findings[-20:]

['edule to an AM schedule to a PM sch',
 'rly get XA00 or XB00 AM short calls and am a',
 'hat wakes me at XA00 AM rested to work all d',
 ' prepare for an XA15 AM show time with an XX',
 '30 AM van at the ZZZ hotel',
 'cond leg of an early AM flight).We were flig',
 'ing from PM trips to AM trips and was having',
 "g 'NAV'.$I was on an AM reserve schedule fro",
 'ed again for a XB:40 AM report the next day?',
 'dy for duty at XA:00 AM only to switched to ',
 'wed by a report at 6 AM on day 4.  Even with',
 'EARANCE. AS FAR AS I AM AWARE; THERE WAS NO ',
 'd getting up at XA00 AM and driving 1 hour t',
 ' trips required XA00 AM get-up. I recognize ',
 'rom midnight to 6:30 AM local time. This pro',
 'e the error. I am an AM Reserve pilot but ha',
 'n - similar to a bad AM signal) was so sever',
 'r and NOTAMS at XA30 AM prior to heading out',
 'ive for work at XZ20 AM local; and fly all n',
 'asus computer. The 2 AM fatigue certainly co']

Mostly time indications. We will subsitute and accept that some entries (e.g. am = I am) will be processed wrongly.

#### Distance in MI (miles)

In [ ]:
# Compile RegEx
r_dist_MI = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \d                            # at least one digit, sticked to 'mi'
                  MI
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_dist_MI.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 7 matches


this RegEx matches also '20minutes' --> by replacing by ' mi' (without a space at the end), we leave the differentiation task to the tokenizer

In [ ]:
findings[:10]

[' Someone did about 20min later. I was speci',
 'equent two blocks (45minutes). Traffic Mana',
 ' widespread VFR in 10mi haze and ATIS repor',
 'iscretion. I waited 2minutes; and then bega',
 'they were about 1hr 5min in the skill check',
 '; so 1hr 40min total for the trai',
 ' 800 lbs less than 45min reserve and fuel t']

#### Distance in SM (statute miles)

replace by ' SM ' 

In [ ]:
# Compile RegEx
r_dist_SM = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \d                            # at least one digit, sticked to 'mi'
                  SM
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_dist_SM.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 12 matches


In [ ]:
findings[:10]

['ility greater than 10SM.  We had been clear',
 'el. METAR 150/7G14 10SM -RA OVC019 A2982.$A',
 'T FMZZXA00 VRB05KT P6SM VCSH SCT025 BKN0301',
 'e of arrival was... 4SM -RA BR OVC015We pla',
 'IAD [METAR] 00000KT 2SM  -RA  BR  BKN007  O',
 'eeing the runway at 8sm; but they could at ',
 'y. The response was 1SM in dust with winds ',
 ' in low visibility (2SM RA) my GPS lost pos',
 'icating 01014G22KT 10SM BKN009.Aircraft X c',
 'ity was OVC033 and 10SM. Weather at closest']

#### Distance in NM (nautical miles)

replace by ' NM ' 

In [ ]:
# Compile RegEx
r_dist_NM = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \d                            # at least one digit, sticked to 'mi'
                  NM
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_dist_NM.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 46 matches


In [ ]:
findings[:10]

[' an approximately 2.5NM final I heard the h',
 ' an approximately 1.5NM final I turned my a',
 ' were still about 1.5NM out and approximate',
 'R. As I reached the 4nm distance; the 152 c',
 '72 as there is only 1NM that separates them',
 'arby airport; only 90NM away. The flight la',
 '1100 scattered. At 15NM west of IOW at 500-',
 'at technically from 1nm out that airspace b',
 'mum separation of 1-2nm should be instated ',
 'pushing it north by 5nm to deconflict this ']

#### MIN - minutes

replace by ' min'

In [ ]:
# Compile RegEx for MIN preceeded by a number
# There are much more entries containing a space between the number and 'min'
# We are interested in those where the number sticks to 'min' or 'minutes'

# When using OR, you have to put the superset 'minutes' before 'minute' before 'min'
# If you use the opposite order, it will find only 'min'
r_MIN = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  #.{0,20}                       # show up to 20 characters before the pattern
                  \d+                           # at least one digit. Without '+', it maches only '0minute' from '20minutes'
                  MINUTES                        
                  |
                  \d+
                  MINUTE
                  |
                  \d+
                  MINS
                  | 
                  \d+
                  MIN
                  #.{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_MIN.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 6 matches


In [ ]:
findings[:20]

['20min', '45minutes', '2minutes', '5min', '40min', '45min']

In [ ]:
# Compile RegEx
r_MINUTE = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  MINUTES#.{0,20}
                  |.{0,20}
                  MINUTE     
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)
findings = r_MINUTE.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 942 matches


In [ ]:
# Compile RegEx
r_MINUTE = re.compile(r"""
                  #.{0,20}
                  \d+
                  #\D
                  #.{0,20}
                """, re.VERBOSE)
findings = r_MINUTE.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 24367 matches


In [ ]:
findings[:10]

['80', '110', '30', '110', '85', '80', '24', '8', '24', '6']

Note: the RegEx expression 

```
(MINUTE|MINUTES)
```
does not return any entry 'minutes' because it finds the subset 'minute' within 'minutes' first.

On the other hand, using
```
(MINUTES|MINUTE)
```
returns both 'minutes' and 'minute'.


**Do we want to substitute 'minutes' in 'few minutes' ?
No, right?**

#### HR

replace by ' hr'

In [ ]:
# Compile RegEx for HR
r_HR = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \d+
                  HR                        
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_HR.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 3 matches


In [ ]:
findings[:10]

['se to the end of my 12hr shift and I should ',
 'and they were about 1hr 5min in the skill c',
 'heck; so 1hr 40min total for the']



*   'HR' is almost always preceeded by a number and indicates a time duration





#### LBS

replace by ' LBS '

Better: search for 'LBS' instead of 'LB' because the unit always appears in plural.

This also avoids matching 'aLBeit', 'taiLBoom'

In [ ]:
# Compile RegEx for MB
r_LBS = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \d+
                  LBS                        
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_LBS.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 8 matches


In [ ]:
findings[:10]

['ight to be above 38;500lbs. Our flight load fo',
 ' buffer; planned at 2099lbs; legally required u',
 'd the fueler to add 300lbs of fuel and 300lbs ',
 'Manifest that was 1;103lbs over the weights us',
 'auge was indicating 500lbs of fuel and the rig',
 ' gauges resulted in 700lbs reading when fuel w',
 'patch add another 1;000lbs fuel. Eventually ha',
 'us arriving with 11;000lbs. FOD. [ZZZ] ATIS wa']

#### MB

In [ ]:
# Compile RegEx for MB
r_MB = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \d+
                  \s*
                  MB                        
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_MB.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 0 matches


In [ ]:
findings[:10]

[]

**Most matches come from 'cliMB', 'remeMBer', etc.**

There are ~30 matches in all narratives for \d+ \s* MB --> ignore this unit

#### KW

Only 2 matches in ALL narratives --> we ignore it

In [ ]:
# Compile RegEx
r_KW = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \d+
                  KW                      
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_KW.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 0 matches


In [ ]:
findings[:10]

[]

#### KM

Only 3 matches in ALL narratives ---> ignore it

In [ ]:
# Compile RegEx
r_KM = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \d+
                  KM                      
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_KM.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 0 matches


In [ ]:
findings[:10]

[]

#### dB

Only 4 matches in ALL narratives ---> ignore it

In [ ]:
# Compile RegEx
r_dB = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \d+
                  dB                      
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_dB.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 0 matches


In [ ]:
findings[:10]

[]

#### KG

Only 30 matches in ALL narratives ---> ignore it

In [ ]:
# Compile RegEx
r_KG = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \d+
                  KG                      
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_KG.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 0 matches


In [ ]:
findings[:10]

[]

#### deg

We have treated the 'degrees C' under Celcius
The rest is the inherent limitation of the BagOfWords approach.

In [ ]:
# Compile RegEx
r_deg = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  #\d+
                  deg                        
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_deg.findall(all_narr)
print(f"Found {len(findings)} matches")

Found 548 matches


In [ ]:
findings[:100]

['limbing turn to 180 degrees continue climb ',
 'turning left to 230 degrees commencing a 5 ',
 'n is he was on a 45 degree downwind entry f',
 'flap handle from 20 degrees to 25 degrees a',
 'nd then to 30 degrees while the flap ',
 'eedle stopped at 20 degrees and the Master ',
 'ive minutes: 1) Ten degree course change re',
 'lting in a 15 to 20 degree heading change d',
 'ng of two zero zero degrees for our climb. ',
 ' it causes a severe degradation in safety a',
 ' changed to 7 vs 15 degrees of bank. This d',
 ' turn approaches 90 degrees. The flight con',
 'onics defaults to 7 degrees after a power c',
 ' to also reset to 7 degrees of bank which a',
 'ercept angle was 80 degrees; not the standa',
 'rd 30 degrees one would expec',
 '. I initiated a 180-degree reversal and des',
 'ld zero roll at 30+ degrees; although we we',
 're at about 30-40 degrees right bank.  Be',
 'o we moved about 15 degrees left; to about ',
 'nd; approaching 180 degrees to the runway d',
 'rthward afte

#### IF AND SO

In [ ]:
# Replace "IF" by " <IF> "  as <IF> will stand for "Intermediate Fix" expression 
# Compile RegEx for "Intermediate Fix" abbreviation
r_IF = re.compile(r"""
                  .{0,20}                       # show up to 20 characters before the pattern
                  [\s\.\,\;\:\/]
                  IF
                  [\s\.\,\;\:\/]
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_IF.findall(all_narr)
print(f"Found {len(findings)} matches")
findings[:20]

Found 9 matches


['ht and realized that IF he followed the TCAS',
 'n active runway; not IF. I hope that we lear',
 '500 feet back to the IF/MM on the approach. ',
 'noted in the system; IF a cargo-spill-relate',
 'ok that for RODDE an IF on that same approac',
 '. 1st and [foremost] IF OPS HAS TROUBLE WITH',
 'IC SYSTEM COMPONENT. IF YOU DO NOT DO THIS; ',
 'f 2000 feet. Passing IF; I hit APPR mode and',
 'enu because it is an IF; not and IAF. I told']

In [ ]:
# Replace "SO" by " <SO> "  as <SO> will stand for "Second Officer" expression 
# Compile RegEx 
r_SO = re.compile(r"""
                  .{0,20}                       # show up to 20 characters before the pattern
                  [\s\.\,\;\:\/]
                  SO
                  [\s\.\,\;\:\/]
                  .{0,20}                       # show up to 20 characters after the pattern 
                """, re.VERBOSE)

findings = r_SO.findall(all_narr)
print(f"Found {len(findings)} matches")
findings[:20]

Found 0 matches


[]

DO NOT REPLACE : mostly 'So' as common word + very few occurences

#### Substitute function

In [ ]:
# Compile Regular Expressions
r_R_L_C = re.compile(r"""
                  (?i)                   # turns on the case-insensitive mode of RegEx
                  \d+R 
                  | 
                  \d+L                   # at least a number *before* 'R' or 'L'
                  |
                  \d+C
                  """, re.VERBOSE)

r_Right_Left_Center = re.compile(r"""
                  (?i)                   # turns on the case-insensitive mode of RegEx
                  \d+ Right 
                  | 
                  \d+ Left               # at least a number *before*
                  |
                  \d+ Center
                  """, re.VERBOSE)

r_Celcius_neg = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \s                            # Avoid finding an aircraft model type, e.g. 'MH-65C'
                  -                             # Our hypothesis: most temp. indications are negative
                                                # otherwise confustion with RWY indication '36C' (center)
                  \d+                           
                  C
                  \s                            
                """, re.VERBOSE)

r_Celcius_degrees = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \d+
                  \s                           
                  degrees                       # appears mostly in plural form
                  \s* 
                  C
                  \s                            # without this, you will match '120 degree clearing turn' 
                """, re.VERBOSE)

r_Mach_1 = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  Mach
                  \s*
                  0*\.\d{1,}                    # decimal number *after* 'Mach'
                  """, re.VERBOSE)

r_Mach_2 = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  0*\.\d{1,}                    # decimal number *before* 'Mach'
                  \s*
                  Mach
                  """, re.VERBOSE)

r_gust = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \d+
                  g
                  \d+
                  """, re.VERBOSE)

# 0000Z format
r_Z_1 = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \s
                  \d{4,4}                       # exactly 4 digits preceeding Zulu time (UTC +0) 'Z'    
                  Z
                  \s                            
                  """, re.VERBOSE)

# 00:00Z format
r_Z_2 = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \s
                  \d{2,2}                       # exactly 2 digits     
                  \:
                  \d{2,2}
                  Z
                  \s                            
                  """, re.VERBOSE)

r_KTS = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \d+
                  KTS
                  |
                  \d+
                  KT                        
                """, re.VERBOSE)

r_alt_1 = re.compile(r"""
                    (?i)                          # turns on the case-insensitive mode of RegEx
                    FL\d{1,}     
                    """, re.VERBOSE)

r_alt_2 = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \d{0,2}[\.\,\;]
                  \d*
                  FT
                  """, re.VERBOSE)

r_am = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx                 
                  [0-1X\w]{1,2}
                  [\;\:\,\.]*
                  [0-59]{1,2}
                  am                
                  """, re.VERBOSE)

r_pm = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx                 
                  [0-1X\w]{1,2}
                  [\;\:\,\.]*
                  [0-59]{1,2}
                  pm                
                  """, re.VERBOSE)

r_dist_MI = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \d                            # at least one digit, sticked to 'mi'
                  MI
                  """, re.VERBOSE)

r_dist_SM = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \d                            # at least one digit, sticked to 'mi'
                  SM
                  """, re.VERBOSE)

r_dist_NM = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \d                            # at least one digit, sticked to 'mi'
                  NM
                  """, re.VERBOSE)

r_MIN = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \d+                           # at least one digit. Without '+', it maches only '0minute' from '20minutes'
                  MINUTES                        
                  |
                  \d+
                  MINUTE
                  |
                  \d+
                  MINS
                  | 
                  \d+
                  MIN
                  """, re.VERBOSE)

r_HR = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \d+
                  HR                        
                  """, re.VERBOSE)

r_LBS = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \d+
                  LBS                        
                  """, re.VERBOSE)

In [ ]:
# Define replacements to perform
# All the findings of the RegEx's in the list will be substituted by the corresponding key
# Note the precise use a spaces the substitute, to avoid problems during tokenization
my_dict = {
          ' <RUNWAY> ': [r_R_L_C, r_Right_Left_Center],
          ' <TEMP_C_NEG> ': [r_Celcius_neg],
          ' <TEMP_C_POS> ': [r_Celcius_degrees],
          ' Mach ': [r_Mach_1, r_Mach_2],
          ' <GUST> ': [r_gust],
          ' <TIME> ': [r_Z_1, r_Z_2],
          ' KT ': [r_KTS],
          ' FT ': [r_alt_1, r_alt_2],
          ' PM ': [r_pm],
          ' AM': [r_am],   # NO space at the end of ' AM', to avoid comfusion with 'amps'
          ' MI': [r_dist_MI],
          ' SM ': [r_dist_SM],
          ' NM ': [r_dist_NM],
          ' MIN': [r_MIN], # NO space at the end of ' MIN'
          ' HR': [r_HR],   # NO space at the end of ' HR'
          ' LBS ': [r_LBS], 
          }

##### Sandbox

In [ ]:
txt = 'bla bl 18L on an RNAV SID. I w'

In [ ]:
# Loop through the keys of the dict
for k in my_dict.keys():  

  # loop through the list of RegEx's that correspond to that key
  for regex in my_dict[k]:
    #repl_result = regex.sub(k, txt)
    repl_result = re.subn(regex, k, txt)
    print(repl_result[0], "\n\nNumber of replacements:", repl_result[1])

 <RUNWAY>  on an RNAV SID. I w 

Number of replacements: 1
bla bl 18L on an RNAV SID. I w 

Number of replacements: 0
bla bl 18L on an RNAV SID. I w 

Number of replacements: 0
bla bl 18L on an RNAV SID. I w 

Number of replacements: 0
bla bl 18L on an RNAV SID. I w 

Number of replacements: 0
bla bl 18L on an RNAV SID. I w 

Number of replacements: 0
bla bl 18L on an RNAV SID. I w 

Number of replacements: 0
bla bl 18L on an RNAV SID. I w 

Number of replacements: 0
bla bl 18L on an RNAV SID. I w 

Number of replacements: 0
bla bl 18L on an RNAV SID. I w 

Number of replacements: 0
bla bl 18L on an RNAV SID. I w 

Number of replacements: 0
bla bl 18L on an RNAV SID. I w 

Number of replacements: 0
bla bl 18L on an RNAV SID. I w 

Number of replacements: 0
bla bl 18L on an RNAV SID. I w 

Number of replacements: 0
bla bl 18L on an RNAV SID. I w 

Number of replacements: 0
bla bl 18L on an RNAV SID. I w 

Number of replacements: 0
bla bl 18L on an RNAV SID. I w 

Number of replacements:

##### Clean code

In [ ]:
def substitute_RegEx(my_dict):
  """
  Inputs: a dictionnary with RegEx's and subsitutes
  Go through the narratives and replace the findings of the RegEx's by the substitutes 
  passed as input (keys of the dictionnary).
  Write the new version of the narrative after the replacements into a new column of df 
  """
  
  # Time function execution
  import time
  start_time = time.time()
  print(7*'-', "Execution started, why don't you grab a coffee...", 7*'-', '\n')

  # Copy the narratives into a new column
  df['Narrative_RegEx_subst'] = df['Narrative']

  # Initialize counters
  repl_counter = 0
  progress = 0

  # Loop through the narratives using their index (here ACN number)
  for idx in df['Narrative_RegEx_subst'].index[:2000]:
    
    # Loop through the keys of the dict
    for k in my_dict.keys():  
      
      # Loop through the list of RegEx's that correspond to that key
      for regex in my_dict[k]:
        new_term = k
        repl_result = re.subn(regex, new_term, df['Narrative_RegEx_subst'].loc[idx])
        # The re.subn() method returns the new version of the target string after the replacements 
        # The second element is the number of replacements it has made
    
        # New version of the narrative, after the replacements
        df['Narrative_RegEx_subst'][idx] = repl_result[0]

        # Increment the counter of remplacements by the number of replacements done in the narrative
        repl_counter = repl_counter + repl_result[1]
      
    # Report on the execution progress  
    progress += 1
    if progress % 1000 == 0:
      print(f"{progress} narratives processed; {repl_counter} replacements done so far \n")

  end_time = time.time()
  print(f"--- Executed in {np.round((end_time - start_time)/60,1)} minutes ---")
  print(f"{repl_counter} replacements in total")

  return None

In [ ]:
# Call the function
# /!\ takes time to execute!
substitute_RegEx(my_dict)

------- Execution started, why don't you grab a coffee... ------- 



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



 1000 narratives processed; 773 replacements done so far

 2000 narratives processed; 1513 replacements done so far
--- Executed in 2.5 minutes ---
1513 replacements in total


In [ ]:
# Write all the entries of 'Anomaly' into a single string using 
# a delimiter which is not present in the data, i.e. '$'
all_narr_subst = ''
all_narr_subst = '$'.join(str(entry) for entry in df['Narrative_RegEx_subst'][:5000]) # do not include a ' ' before / after the '$' !

KeyError: ignored

In [ ]:
# Verify the replacements
r_Mach = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  \sMach\s
                  .{0,20}                       # show up to 20 characters after the pattern
                  """, re.VERBOSE)
findings = r_Mach.findall(all_narr_subst)
print(f"Found {len(findings)} matches")
findings[:10]

Found 44 matches


['ling at an indicated Mach speed of 0.817. We t',
 "; a squawk code; and mach decimal seven zero;'",
 ' original speed was  Mach  and new Economy spe',
 "ed was  Mach ; therefore I didn't",
 ' that the autopilot; Mach trim and yaw damper ',
 'gaged the autopilot; Mach trim and yaw damper.',
 'ost his airspeed and mach indications. The Cap',
 'll having Air Speed; Mach and altimeter issues',
 ' speed was noted at  Mach ; 340 KTS. I immedia',
 'ally light wave. Our mach went from .82 to .85']

In [ ]:
# Verify the replacements
r_Runway = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  <RUNWAY>
                  .{0,20}                       # show up to 20 characters after the pattern
                  """, re.VERBOSE)
findings = r_Runway.findall(all_narr_subst)
print(f"Found {len(findings)} matches")
findings[:10]

Found 2030 matches


['was VMC and we setu <RUNWAY>  backed up by the I',
 'r; flaps 25; bleeds <RUNWAY> . These takeoffs ar',
 'raffic Controller.$ <RUNWAY>  on an RNAV SID. I ',
 'path of a departing <RUNWAY> ct we stopped the t',
 ' HSI and the need t <RUNWAY> $A pilot in our fly',
 'It is dangerous!!!$ <RUNWAY> ; the F/O (First Of',
 ' threatening.$Night <RUNWAY>  at TUS.  First Off',
 'dinary."$We apparen <RUNWAY>  without clearance.',
 'll was normal. As w <RUNWAY>  we switched to Tow',
 '. There were other  <RUNWAY>  <RUNWAY>  was clos']

In [ ]:
# Verify the replacements
r_Runway = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  .{0,20}                       # show up to 20 characters before the pattern
                  TEMP_C_POS
                  .{0,20}                       # show up to 20 characters after the pattern
                  """, re.VERBOSE)
findings = r_Runway.findall(all_narr_subst)
print(f"Found {len(findings)} matches")
findings[:10]

Found 8 matches


['e Right Inboard br <TEMP_C_POS>  Right Outboard wa',
 'g.) C and the righ <TEMP_C_POS>  from that when it',
 ' at 40; climbed to <TEMP_C_POS> ; the temperature ',
 ' arm. The outside  <TEMP_C_POS> e advanced to 90% ',
 'ay required 45% N1 <TEMP_C_POS>  significant hazar',
 'eft engine was ind <TEMP_C_POS> .We requested a la',
 ' Temperatures that <TEMP_C_POS>  ramp. The APU fai',
 'ticed the temperat <TEMP_C_POS>  the Engine IPS an']